In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

In [11]:
###########################################################
# Define DEFT Module Components
###########################################################

class LocalizationNetwork(nn.Module):
    def __init__(self, input_channels):
        super(LocalizationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, kernel_size=5)

        # Placeholder fc1 (will update after shape calculation)
        self.fc1 = nn.Linear(1, 32)  # Temporary placeholder
        self.fc2 = nn.Linear(32, 6)  # 6 affine parameters

        # Initialize weights for identity transformation
        self.fc2.weight.data.zero_()
        self.fc2.bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically compute the flattened size
        if not hasattr(self, 'computed_fc1'):
            flattened_size = x.view(x.shape[0], -1).shape[1]
            self.fc1 = nn.Linear(flattened_size, 32).to(x.device)
            self.computed_fc1 = True  # Prevent re-initialization

        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.fc1(x))
        theta = self.fc2(x)
        theta = theta.view(-1, 2, 3)
        return theta

class WeightingModule(nn.Module):
    def __init__(self, sigma=0.5):
        super(WeightingModule, self).__init__()
        self.lambda_param = nn.Parameter(torch.tensor(0.5))
        self.sigma = sigma

    def forward(self, grid):
        dist2 = grid[..., 0]**2 + grid[..., 1]**2
        weight = 1 + self.lambda_param * torch.exp(-dist2 / (2 * self.sigma ** 2))
        return weight.unsqueeze(-1)

class DEFTModule(nn.Module):
    def __init__(self, input_channels, sigma=0.5):
        super(DEFTModule, self).__init__()
        self.localization = LocalizationNetwork(input_channels)
        self.weighting = WeightingModule(sigma)

    def forward(self, x):
        theta = self.localization(x)
        grid = F.affine_grid(theta, x.size(), align_corners=False)
        weight = self.weighting(grid)
        x_transformed = F.grid_sample(x, grid, align_corners=False)

        if x.shape[1] > 1:
            weight = weight.expand(-1, x.shape[2], x.shape[3], x.shape[1]).permute(0, 3, 1, 2)
        else:
            weight = weight.permute(0, 3, 1, 2)

        x_weighted = x_transformed * weight
        return x_weighted


In [12]:
###########################################################
# Load ResNet50 Model for Feature Extraction
###########################################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet50(pretrained=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])  # Remove classification layer
resnet_model.eval().to(device)

# Load DEFT Module
deft_model = DEFTModule(input_channels=3).to(device)  # Input channels = 3 (RGB)
deft_model.eval()

###########################################################
# Define Image Transformation
###########################################################
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

C:\Users\PAWANESH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\PAWANESH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
deft_model

DEFTModule(
  (localization): LocalizationNetwork(
    (conv1): Conv2d(3, 8, kernel_size=(7, 7), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(8, 10, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=4000, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=6, bias=True)
  )
  (weighting): WeightingModule()
)

In [13]:
###########################################################
# Feature Extraction Function with DEFT
###########################################################
def extract_features(image_path, transform, device=device):
    """Extract ResNet50 features from a single image after DEFT transformation."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)  # Convert to tensor and send to GPU

        # Apply DEFT Transformation
        with torch.no_grad():
            image = deft_model(image)  # Pass through DEFT Module
            features = resnet_model(image).squeeze().cpu().numpy()  # Extract ResNet50 features
        
        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None

###########################################################
# Paths to RGB and Optical Flow Directories
###########################################################
rgb_path = "D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04"
flow_u_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u"
flow_v_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v"

label_csv_path = "D:/Datasets/Datasets/EPIC_Kitchen/Label/P01_04.csv"
labels_df = pd.read_csv(label_csv_path)

output_csv = "Features/Feature_P01_04_with_DEFT.csv"


In [14]:
###########################################################
# Extract Features with DEFT & Save
###########################################################
S = 5  # Sampling every 5th frame
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting Features with DEFT"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    flow_u_frame_path = os.path.join(flow_u_path, frame)
    flow_v_frame_path = os.path.join(flow_v_path, frame)
    
    rgb_features = extract_features(rgb_frame_path, transform, device)
    flow_u_features = extract_features(flow_u_frame_path, transform, device)
    flow_v_features = extract_features(flow_v_frame_path, transform, device)
    
    if rgb_features is not None and flow_u_features is not None and flow_v_features is not None:
        combined_features = np.concatenate([rgb_features, flow_u_features, flow_v_features])
        
        frame_number = int(frame.split('_')[-1].split('.')[0])
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]
        
        if not label_row.empty:
            verb_class = label_row.iloc[0]['Verb_class']
            noun_class = label_row.iloc[0]['Noun_class']
            action_class = label_row.iloc[0]['Action_class']
        else:
            verb_class, noun_class, action_class = 0, 0, 0  # Default if no label found
        
        features_list.append([frame, verb_class, noun_class, action_class] + combined_features.tolist())

if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

columns = ["Frame", "Verb_class", "Noun_class", "Action_class"] + [f"Feature_{i}" for i in range(len(combined_features))]
df = pd.DataFrame(features_list, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Feature extraction completed! Saved to {output_csv}")


Extracting Features with DEFT:  50%|████████████████████████▏                       | 636/1262 [00:44<00:33, 18.55it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3161.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3161.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3161.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3161.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3166.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3166.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3166.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3166.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3171.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT:  51%|████████████████████████▍                       | 642/1262 [00:44<00:28, 22.07it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3191.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3191.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3191.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3191.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3196.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_3196.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_3196.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_3196.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_3201.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT:  51%|████████████████████████▋                       | 648/1262 [00:44<00:24, 24.76it/s]

...

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6251.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6251.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6251.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6251.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6256.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6256.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6256.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6256.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6261.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT: 100%|██████████████████████████████████████████████▉| 1260/1262 [01:10<00:00, 24.84it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6276.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6276.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6276.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6276.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6281.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6281.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6281.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6281.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6286.jpg due to error: [Errno 2] No such file or directory:

Extracting Features with DEFT: 100%|███████████████████████████████████████████████| 1262/1262 [01:10<00:00, 17.80it/s]


Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6301.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6301.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6301.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6301.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_6306.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_6306.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_6306.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_6306.jpg'
Feature extraction completed! Saved to Features/Feature_P01_04_with_DEFT.csv


In [15]:
f=pd.read_csv("Features/Feature_P01_04_with_DEFT.csv")
f

,Frame,Verb_class,Noun_class,Action_class,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,...,Feature_6134,Feature_6135,Feature_6136,Feature_6137,Feature_6138,Feature_6139,Feature_6140,Feature_6141,Feature_6142,Feature_6143
0,frame_0001.jpg,0,0,0,0.651917,0.407176,0.076979,0.357042,1.976730,0.638776,...,0.119569,0.000009,0.029647,0.155487,0.006345,0.000375,0.108428,0.139510,0.029922,0.226609
1,frame_0006.jpg,0,23,0,0.546664,0.483001,0.135680,0.225158,2.126921,0.551493,...,0.157487,0.000000,0.006592,0.087250,0.241418,0.000000,0.131806,0.069871,0.019073,0.259733
2,frame_0011.jpg,0,23,0,0.483140,0.646575,0.202512,0.355270,2.394428,0.560053,...,0.212274,0.019422,0.061624,0.105661,0.028055,0.000000,0.000000,0.177602,0.010195,0.164882
3,frame_0016.jpg,0,23,0,0.404216,0.568184,0.049507,0.525459,2.029208,0.432885,...,0.484715,0.122950,0.030248,0.096550,0.606497,0.011370,0.219069,0.188862,0.002258,0.156196
4,frame_0021.jpg,0,23,0,0.504452,0.529196,0.116372,0.598458,1.932839,0.722018,...,0.611212,0.257860,0.085634,0.485791,0.232887,0.000439,0.003415,0.046124,0.019902,0.666137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,frame_3136.jpg,4,23,14,1.126213,0.175981,1.183292,1.671395,0.540780,0.675402,...,0.249193,0.011448,0.177856,0.290418,0.058419,0.020178,0.000000,0.038382,0.000758,0.110715
628,frame_3141.jpg,4,23,14,0.875532,0.130069,1.162229,1.753934,0.414319,0.467606,...,0.014190,0.004053,0.060821,0.077685,0.000000,0.007218,0.001084,0.041424,0.011284,0.098705
629,frame_3146.jpg,0,0,0,0.990685,0.336301,0.916479,1.528714,0.513048,0.428346,...,0.383098,0.005483,0.029652,0.194086,0.161036,0.001043,0.000000,0.023354,0.198656,0.131415
630,frame_3151.jpg,4,7,15,1.255465,0.304995,1.173652,1.445411,0.552218,0.506052,...,0.182186,0.000100,0.025416,0.243660,0.004754,0.002198,0.000000,0.130777,0.013741,0.037159
